## University of British Columbia LIbrary API Vignette
### Introduction
Something or other other

Something about API access and registering for a private key here:

First, import the package:

In [18]:
from final_project_herman_arielle import final_project_herman_arielle as fp

This next step is optional, as UBC provides a public api key, and the functions in this package utilize this key by default.

In [19]:
bc_api_key = os.getenv("POETRY_BC_LIBRARY_API_KEY")
print(bc_api_key[:3])

897


## Functions
#### 1.  List collections

In [21]:
fp.collections("darwin", "foobar", 5, api_key=bc_api_key)

The status of this request is:  200 . Please wait. 

darwin is not a current collection in the University of British Columbia Library. 
Please check your spelling or check the output to see available collections.
foobar is not a current collection in the University of British Columbia Library. 
Please check your spelling or check the output to see available collections.


TypeError: 'in <string>' requires string as left operand, not int

#### 2. List field structures of a given collection

In [23]:
example = fp.structures("darwin")
example

AttributeError: module 'final_project_herman_arielle.final_project_herman_arielle' has no attribute 'structures'

#### 3. List the differences between two structures of a given collection

In [ ]:
difference_two(example[0], example[1])

In [ ]:
help(collections)